#  Домашняя работа 
Для выполнения домашнего задания необходимо взять файл boston house-prices datase (sklearn.datasets.load_boston) и построить модель.
По аналогии задания к лекции «Классификация: Логистическая регрессия и SVM».
А именно построить модель регрессии, дерево решений, оптимизировать параметры и сравнить результаты.

In [10]:
from sklearn.tree import DecisionTreeRegressor #Дерево решений
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.datasets import load_boston
import pandas as pd

In [2]:
boston = load_boston()

In [3]:
#Описание атрибутов датасета
DESCR = {'CRIM':'Криминальный уровень преступности на душу населения',
         'ZN':'Доля жилой земли, зонированной для участков площадью более 25 000 кв. футов.',
         'INDUS':'Доля акров неторгового бизнеса',
         'CHAS':'Фиктивная переменная (= 1, если тракт ограничивает реку; 0 в противном случае)',
         'NOX':'Концентрация оксидов азота NOX (частей на 10 млн.)',
         'RM':'Cреднее количество комнат в жилом помещении',
         'AGE':'Доля занятых квартир в домах построенных до 1940 года',
         'DIS':'Взвешенные расстояния до пяти Бостонских центров занятости',
         'RAD':'Индекс доступности радиальных магистралей',
         'TAX':'Cтавка налога на недвижимость с полной стоимостью за 10 000 долларов США',
         'PTRATIO':'Cоотношение учащихся и учителей',
         'B':'1000(Bk - 0.63)^2, где Bk-доля чернокожих',
         'LSTAT':'% более населения с низким статусом',
         'MEDV':'Медианная стоимость домов, занятых владельцами, в $1000'
         }

In [4]:
#Общая информация по датасету
data_boston = pd.concat([pd.DataFrame(boston.data, columns = boston.feature_names), 
                         pd.DataFrame({'MEDV':boston.target})], axis = 1)
data_boston.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    float64
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    float64
 9   TAX      506 non-null    float64
 10  PTRATIO  506 non-null    float64
 11  B        506 non-null    float64
 12  LSTAT    506 non-null    float64
 13  MEDV     506 non-null    float64
dtypes: float64(14)
memory usage: 55.5 KB


In [12]:
#Признаковое пространство
data_boston

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48,22.0


In [67]:
#Функция для тестирования модели
def test_model(model_, data):
    data_train, data_test, rez_train, rez_test = train_test_split( data.iloc[:,:-1], 
                                                                   data.iloc[:,-1], 
                                                                   test_size=0.3, 
                                                                   random_state = 42 )
    model_.fit(data_train, rez_train)
    ret = {'Ошибка': round(mean_absolute_error( model_.predict(data_test), 
                                                                   rez_test ), 2),
           'Оценка': round(model_.score(data_test, rez_test), 2)}
    return ret

#Оптимизируем параметры
lst_par_max_depth = [1, 5, 10, 20, 40]
lst_par_criterion = ['mse', 'friedman_mse', 'mae']
rezult = []
for md in lst_par_max_depth:
    for cri in lst_par_criterion:
        dt = DecisionTreeRegressor( criterion = cri, max_depth = md )
        rezult.append( [md, cri, test_model(dt, data_boston)] )
    
rezult


[[1, 'mse', {'Ошибка': 4.83, 'Оценка': 0.42}],
 [1, 'friedman_mse', {'Ошибка': 4.83, 'Оценка': 0.42}],
 [1, 'mae', {'Ошибка': 4.85, 'Оценка': 0.4}],
 [5, 'mse', {'Ошибка': 2.48, 'Оценка': 0.85}],
 [5, 'friedman_mse', {'Ошибка': 2.49, 'Оценка': 0.85}],
 [5, 'mae', {'Ошибка': 2.47, 'Оценка': 0.81}],
 [10, 'mse', {'Ошибка': 2.64, 'Оценка': 0.74}],
 [10, 'friedman_mse', {'Ошибка': 2.24, 'Оценка': 0.87}],
 [10, 'mae', {'Ошибка': 2.8, 'Оценка': 0.73}],
 [20, 'mse', {'Ошибка': 2.55, 'Оценка': 0.83}],
 [20, 'friedman_mse', {'Ошибка': 2.4, 'Оценка': 0.85}],
 [20, 'mae', {'Ошибка': 2.97, 'Оценка': 0.7}],
 [40, 'mse', {'Ошибка': 2.56, 'Оценка': 0.85}],
 [40, 'friedman_mse', {'Ошибка': 2.41, 'Оценка': 0.85}],
 [40, 'mae', {'Ошибка': 3.08, 'Оценка': 0.64}]]